In [28]:
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD, RDFS, DCTERMS #most common namespaces
import urllib.parse #for parsing strings to URI's
import numpy as np
from array import array

In [29]:
g = Graph()
title = Namespace('http://example.org/title/')
author = Namespace('http://example.org/author/')
year = Namespace('http://example.org/year/')
doi = Namespace('http://example.org/doi/')
pmid = Namespace('http://example.org/PMID/')
journal = Namespace('http://example.org/journal/')
volume = Namespace('http://example.org/volume/')
abstract = Namespace('http://example.org/abstract/')
keyword_abstract = Namespace('http://example.org/keyword_abstract/')
keyword_mesh = Namespace('http://example.org/keywords_mesh/')
keyword_merged = Namespace('http://example.org/keywords_merged/')
links = Namespace('http://example.org/fulltext_links/')
articles = Namespace('http://example.org/related_articles/')
fields = Namespace('http://example.org/fields/')
subfields = Namespace('http://example.org/subfield/')
aimethod = Namespace('http://example.org/ai_method')

In [30]:
df = pd.read_json("extractUntil2020.json")
df.to_csv()
df.head(5)

,Title,Authors,Year,Doi,PMID,Journal,Volume,Abstract,Keywords_Abstract,Keywords_MeSH,Keywords_Merged,FullTextLinks,Related Articles
0,Clipping in neurocontrol by adaptive dynamic p...,Michael Fairbank; Danil Prokhorov; Eduardo Alo...,2014,10.1109/TNNLS.2014.2297991,25291742,IEEE Trans Neural Netw Learn Syst,25,"In adaptive dynamic programming, neurocontrol...",,Algorithms; Artificial Intelligence*; Comput...,Algorithms; Artificial Intelligence*; Comp...,[https://dx.doi.org/10.1109/TNNLS.2014.2297991],"19842990,25751878,23954546,19386468,24995352"
1,A scalable FPGA-based cerebellum for passage-o...,Junwen Luo; Graeme Coapes; Terrence Mak; Tadas...,2014,10.1109/EMBC.2014.6944279,25570647,Conf Proc IEEE Eng Med Biol Soc,2014,The cerebellum plays a critical role for sens...,,"Research Support, Non-U.S. Gov't; Animals; A...","Research Support, Non-U.S. Gov't; Animals; ...",[https://dx.doi.org/10.1109/EMBC.2014.6944279],"26452290,29115280,17953620,19495900,18807937"
2,Epileptic EEG classification based on kernel s...,Qi Yuan; Weidong Zhou; Shasha Yuan; Xueli Li; ...,2014,10.1142/S0129065714500154,24694170,Int J Neural Syst,24,The automatic identification of epileptic EEG...,,"Research Support, Non-U.S. Gov't; Artificial ...","Research Support, Non-U.S. Gov't; Artificia...",[https://www.worldscientific.com/doi/10.1142/S...,"30076538,25653073,21616643,27458376,16013270,3..."
3,MS2CNN: predicting MS/MS spectrum based on pro...,Yang-Ming Lin; Ching-Tai Chen; Jia-Ming Chang;,2019,10.1186/s12864-019-6297-6,31874640,BMC Genomics,20,Background: Tandem mass spectrometry allows...,Deep convolutional neural networks; Deep lea...,Evaluation Study; Deep Learning; HeLa Cells;...,Deep convolutional neural networks; Deep lea...,[https://bmcgenomics.biomedcentral.com/article...,"29125736,31847456,23695934,22610447,30417778"
4,Ultrasound Image Segmentation: A Deeply Superv...,Deepak Mishra; Santanu Chaudhury; Mukul Sarkar...,2019,10.1109/TBME.2018.2877577,30346279,IEEE Trans Biomed Eng,66,Objective: Segmentation of anatomical struc...,,"Research Support, Non-U.S. Gov't; Blood Vesse...","Research Support, Non-U.S. Gov't; Blood Ves...",[https://dx.doi.org/10.1109/TBME.2018.2877577],"30136285,31356693,31074513,32030668,28445774,3..."


In [31]:
algorithms = pd.read_csv("Book1.csv", sep=';')
algorithms.head(30)


regression = algorithms['regression'].values
instance_based = algorithms['instance based'].values
#regularization = algorithms['regularization'].values
decision_tree = algorithms['decision tree'].values
bayesian = algorithms['bayesian'].values
clustering = algorithms['clustering'].values
association_rule = algorithms['association rule learning'].values
ann = algorithms['artificial neural network'].values
deep_learning = algorithms['deepl learning'].values
dimensionality_reduction = algorithms['dimensionality reduction'].values
ensemble_algorithm = algorithms['ensemble algorithm'].values
regularization = ['Ridge Regression','Least Absolute Shrinkage and Selection Operator','Elastic Net','Least-Angle Regression','LASSO','LARS']

In [32]:
for index, row in df.iterrows():
    
    authors = str(row['Authors']).split(";")
    for a in authors[:-1]:
        g.add((URIRef(pmid+str(row['PMID'])), DCTERMS['creator'], Literal(a, datatype=XSD.string) ))
        
    keywords = str(row['Keywords_Merged']).split(";")
    for key in keywords:
        g.add((URIRef(pmid+str(row['PMID'])), URIRef(keyword_merged+'key_merged'), Literal(key, datatype=XSD.string) ))
        
    fullTexts = str(row['FullTextLinks']).split(";")
    for text in fullTexts:
        g.add((URIRef(pmid+str(row['PMID'])), RDFS.comment, Literal(text, datatype=XSD.string) ))
        
    related = str(row['FullTextLinks']).split(";")
    for a in related:
        g.add((URIRef(pmid+str(row['PMID'])), RDFS.seeAlso, Literal(a, datatype=XSD.string) ))
    
    g.add((URIRef(pmid+str(row['PMID'])),RDF.type, Literal(row['PMID'])))
    g.add((URIRef(pmid+str(row['PMID'])), DCTERMS['identifier'], Literal(row['PMID']))) 
    g.add((URIRef(doi+str(row['PMID'])), DCTERMS['identifier'],Literal(row['Doi'], datatype=XSD.integer) ))
    g.add((URIRef(pmid+str(row['PMID'])), DCTERMS['publisher'], Literal(row['Journal'], datatype=XSD.string) ))
    #g.add((URIRef(pmid+str(row['PMID'])), URIRef(volume+'volume'), Literal(row['Volume'], datatype=XSD.string) ))
    g.add((URIRef(pmid+str(row['PMID'])), DCTERMS['abstract'], Literal(row['Abstract'], datatype=XSD.string) ))
    #g.add((URIRef(pmid+row['PMID']), URIRef(field+'field'), Literal(row['Field'], datatype=XSD.string) ))
    #g.add((URIRef(pmid+row['PMID']), RDFS.subclassOf, Literal(row['Subfield'], datatype=XSD.string) ))
    #g.add((URIRef(pmid+row['PMID']), URIRef(aimethod+'aimethod'), Literal(row['ai_method'])))
    #g.add((URIRef(loc+urllib.parse.quote(row['Address'])), URIRef(schema+'name'), Literal(row['Address'], datatype=XSD.string) ))
#    for x in row[ai_method]:
#        if x in regression :
#            g.add((URIRef(pmid+str(row['PMID'])), URIRef(title+'regression'), Literal(x))) 
#        elif x in instance_based :
#            g.add((URIRef(pmid-str(row['PMID'])), URIRef(title+'instance_based'), Literal(x)))
#        elif x in regularization :
#            g.add((URIRef(pmid-str(row['PMID'])), URIRef(title+'instance_based'), Literal(x)))
#        elif x in decision_tree:
#            g.add((URIRef(pmid-str(row['PMID'])), URIRef(title+'instance_based'), Literal(x)))
#        elif x in bayesian:
#            g.add((URIRef(pmid-str(row['PMID'])), URIRef(title+'instance_based'), Literal(x)))
#        elif x in clustering:
#            g.add((URIRef(pmid-str(row['PMID'])), URIRef(title+'instance_based'), Literal(x)))
#        elif x in association_rule:
#            g.add((URIRef(pmid-str(row['PMID'])), URIRef(title+'instance_based'), Literal(x)))
#        elif x in ann:
#            g.add((URIRef(pmid-str(row['PMID'])), URIRef(title+'instance_based'), Literal(x)))
#        elif x in deep_learning:
#            g.add((URIRef(pmid-str(row['PMID'])), URIRef(title+'instance_based'), Literal(x)))
#        elif x in dimensionality_reduction:
#            g.add((URIRef(pmid-str(row['PMID'])), URIRef(title+'instance_based'), Literal(x)))
#        elif x in ensemble_algorithm:
#            g.add((URIRef(pmid-str(row['PMID'])), URIRef(title+'instance_based'), Literal(x)))
#        else:
#            g.add((URIRef(pmid-str(row['PMID'])), URIRef(title+'instance_based'), Literal('x')))

    

NameError: name 'ai_method' is not defined

In [33]:
print(g.serialize(format='turtle').decode('UTF-8'))

@prefix ns1: <http://example.org/keywords_merged/> .
@prefix ns2: <http://purl.org/dc/terms/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/PMID/25291742> a 25291742 ;
    ns1:key_merged "  "^^xsd:string,
        "  Algorithms"^^xsd:string,
        "  Artificial Intelligence*"^^xsd:string,
        "  Computer Simulation"^^xsd:string,
        "  Humans"^^xsd:string,
        "  Neural Networks, Computer*"^^xsd:string,
        "  Nonlinear Dynamics*"^^xsd:string,
        "  Reinforcement, Psychology*"^^xsd:string ;
    ns2:abstract " In adaptive dynamic programming, neurocontrol, and reinforcement learning, the objective is for an agent to learn to choose actions so as to minimize a total cost function. In this paper, we show that when discretized time is used to model the motion of the agent, it can be very important to do clipping on the motion of the agent in the final time step of the trajectory. By cl

In [34]:
q = "select ?title ?abstract where {?title ?abstract <http://example.org/PMID/24694170>}"
x = g.query(q)
t = list(x)
print("----------------------------------------------------------------------------------------------")
for y in range(0,len(t)):
    ss=t[y][0]
    #search for id
    q = "select ?relation ?part  where {<"+ss+"> ?relation ?part}"
    xx = g.query(q)
    print(xx)

----------------------------------------------------------------------------------------------


In [35]:
# <id, attribute, value>
#search PMID in the searh bar, and return the data with this PMID, and show its attribute and value, for example below PMID=31946810，and have 12 attributes and values
#<a,?,?>
q = "select ?attribute ?value where {<http://example.org/PMID/31946810>?attribute ?value}"
x = g.query(q)
t = list(x)
print(len(t))  
  
for i in t:
    print(i)
    print("----------------------------------------------------------------------------------------------")

0


In [36]:
# <id, attribute, value>
#search relationship/attribute title in the searh bar, you can change the other attribute
#we can see all the dataset is 79 data which have title
# <?,b,?>
q = "select ?id ?value where {?id <http://example.org/title/title> ?value}"  
x = g.query(q)
t = list(x)
print(len(t))

for i in t:
    print(i)
    print("----------------------------------------------------------------------------------------------")

0


In [37]:
# search value of title in the search bar, for example, search title value is 29993442, but can only do the exactly search
# <id, attribute, value>
# <?,?,c>
q = "select * where {?id <http://example.org/title/title> 29993442 }"  
x = g.query(q)
t = list(x)
print(len(t))   # only one data match this title value
print(t[0])

for i in range(0,len(t)):
    value = str(t[i]).replace("(rdflib.term.URIRef('","").replace("'),)","")  #extract the PMID URI, and then from PMID get all the data
    print(value)
    q = "select * where {<"+ value +"> ?attribute ?value}"  
    x = g.query(q)
    b = list(x)
    if (len(b) > 0) :
        for y in range(0,len(b)):
            print(str(b[y]))
            print("new attibute-------------------------------------------------------------")
        print("\n")
        print("new data-------------------------------------------")

0


IndexError: list index out of range

In [38]:
pip install sparqlwrapper

Note: you may need to restart the kernel to use updated packages.


In [39]:
from SPARQLWrapper import SPARQLWrapper
# maybe something can use in this package, but I don't use this below 

In [40]:
# Above we can only do the exactly search, which can only get all the matched data with the exact vaule we know
#but we can not do the fuzzy query, and below try the fuzzy query
#when search 31 in the search bar, what will happen?
# <id, attribute, value>
q = "select ?id where {?id ?attribute ?value filter regex(?value, '31')}"
x = g.query(q)
t = list(x)
for i in range(0,len(t)):
    value = str(t[i]).replace("(rdflib.term.URIRef('","").replace("'),)","")
    q = "select * where {<"+ value +"> ?attribute ?value}"          # also get the PMID first and then get all the matched data
    x = g.query(q)
    b = list(x)
    if (len(b) > 0) :
        for y in range(0,len(b)):
            print(str(b[y]))
            print("new attibute-------------------------------------------------------------")
        print("\n")
        print("new data-------------------------------------------")

#we can see that as long as "31" appeare in the value, no matter what attibute, it will match.
#but when I want to search for the exact attibute 'title' which may have something like "**31**", I failed, I can only do with all attibutes.

In [41]:
# and maybe you can change the title to other attibutes and also try the fuzzy query.